In [1]:
import pandas as pd
from pandas import json_normalize
import numpy as np
import json as js
import ast as ast
import re

In [2]:
games_org = pd.read_json('output_steam_games.json',lines=True)
games = games_org.copy()

In [3]:
list_rev = []
archivo1 = r'australian_user_reviews.json'
with open(archivo1, encoding='utf-8') as file:
    for line in file.readlines():
        list_rev.append(ast.literal_eval(line))

reviews_org = pd.DataFrame(list_rev)

reviews = reviews_org.copy()
reviews = reviews.explode('reviews',ignore_index=True)

In [4]:
list_items = []
archivo2 = r'australian_users_items.json'
with open(archivo2, encoding='utf-8') as file2:
    for line2 in file2.readlines():
        list_items.append(ast.literal_eval(line2))

items_org = pd.DataFrame(list_items)

items = items_org.copy()
items = items.explode('items',ignore_index=True)

#### Revision de Games

In [120]:
games

,publisher,genres,app_name,title,release_date,tags,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],4.99,0.0,761140.0,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla...",Free To Play,0.0,643980.0,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla...",Free to Play,0.0,670290.0,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,2017-12-07,"[Action, Adventure, Casual]",[Single-player],0.99,0.0,767400.0,彼岸领域
88314,None,None,Log Challenge,None,None,"[Action, Indie, Casual, Sports]","[Single-player, Full controller support, HTC V...",2.99,0.0,773570.0,None
...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,2018-01-04,"[Strategy, Indie, Casual, Simulation]","[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,2018-01-04,"[Strategy, Indie, Casual]","[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,2018-01-04,"[Indie, Simulation, Racing]","[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...","[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"


120444 registros de games antes de borrar duplicados. Son 88309 registros duplicados y vacios. 3135 registros

In [119]:
games.drop(columns=['url','reviews_url'],inplace=True)

In [116]:
games.drop(games[games.duplicated(subset=['publisher', 'app_name', 'title', 'release_date', 'price', 'early_access', 'id',
       'developer'],keep=False)].index,inplace=True)

In [120]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 11.9+ MB


In [39]:
games['early_access'].value_counts()

early_access
0.0    30188
1.0     1947
Name: count, dtype: int64

#### Revision de Reviews

In [94]:
reviews

,user_id,reviews
0,76561197970982479,"{'funny': '', 'posted': 'Posted November 5, 20..."
1,76561197970982479,"{'funny': '', 'posted': 'Posted July 15, 2011...."
2,76561197970982479,"{'funny': '', 'posted': 'Posted April 21, 2011..."
3,js41637,"{'funny': '', 'posted': 'Posted June 24, 2014...."
4,js41637,"{'funny': '', 'posted': 'Posted September 8, 2..."
...,...,...
59328,76561198312638244,"{'funny': '', 'posted': 'Posted July 10.', 'la..."
59329,76561198312638244,"{'funny': '', 'posted': 'Posted July 8.', 'las..."
59330,LydiaMorley,"{'funny': '1 person found this review funny', ..."
59331,LydiaMorley,"{'funny': '', 'posted': 'Posted July 20.', 'la..."


In [93]:
reviews.drop(columns=['user_url'],inplace=True)

In [8]:
reviews.iloc[1,1]

{'funny': '',
 'posted': 'Posted July 15, 2011.',
 'last_edited': '',
 'item_id': '22200',
 'helpful': 'No ratings yet',
 'recommend': True,
 'review': "It's unique and worth a playthrough."}

In [95]:
reviews_expanded = pd.json_normalize(reviews['reviews'])
reviews_expanded=reviews_expanded.replace('',None)

In [101]:
reviews_expanded.head()

,funny,posted,last_edited,item_id,helpful,recommend,review
0,None,"Posted November 5, 2011.",None,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,None,"Posted July 15, 2011.",None,22200,No ratings yet,True,It's unique and worth a playthrough.
2,None,"Posted April 21, 2011.",None,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,None,"Posted June 24, 2014.",None,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,None,"Posted September 8, 2013.",None,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [97]:
reviews = reviews.join(reviews_expanded)

In [99]:
reviews.drop(columns=['reviews'],inplace=True)

In [100]:
reviews.isnull().sum()

user_id            0
funny          51182
posted            28
last_edited    53193
item_id           28
helpful           28
recommend         28
review            58
dtype: int64

In [103]:
reviews

,user_id,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,None,"Posted November 5, 2011.",None,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,None,"Posted July 15, 2011.",None,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,None,"Posted April 21, 2011.",None,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,None,"Posted June 24, 2014.",None,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,None,"Posted September 8, 2013.",None,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...,...
59328,76561198312638244,None,Posted July 10.,None,70,No ratings yet,True,a must have classic from steam definitely wort...
59329,76561198312638244,None,Posted July 8.,None,362890,No ratings yet,True,this game is a perfect remake of the original ...
59330,LydiaMorley,1 person found this review funny,Posted July 3.,None,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
59331,LydiaMorley,None,Posted July 20.,None,730,No ratings yet,True,:D


In [119]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59333 entries, 0 to 59332
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      59333 non-null  object
 1   funny        8151 non-null   object
 2   posted       59305 non-null  object
 3   last_edited  6140 non-null   object
 4   item_id      59305 non-null  object
 5   helpful      59305 non-null  object
 6   recommend    59305 non-null  object
 7   review       59275 non-null  object
dtypes: object(8)
memory usage: 3.6+ MB


#### Revision de Items

In [107]:
items

,user_id,items_count,steam_id,items
0,76561197970982479,277,76561197970982479,"{'item_id': '10', 'item_name': 'Counter-Strike..."
1,76561197970982479,277,76561197970982479,"{'item_id': '20', 'item_name': 'Team Fortress ..."
2,76561197970982479,277,76561197970982479,"{'item_id': '30', 'item_name': 'Day of Defeat'..."
3,76561197970982479,277,76561197970982479,"{'item_id': '40', 'item_name': 'Deathmatch Cla..."
4,76561197970982479,277,76561197970982479,"{'item_id': '50', 'item_name': 'Half-Life: Opp..."
...,...,...,...,...
5170010,76561198329548331,7,76561198329548331,"{'item_id': '373330', 'item_name': 'All Is Dus..."
5170011,76561198329548331,7,76561198329548331,"{'item_id': '388490', 'item_name': 'One Way To..."
5170012,76561198329548331,7,76561198329548331,"{'item_id': '521570', 'item_name': 'You Have 1..."
5170013,76561198329548331,7,76561198329548331,"{'item_id': '519140', 'item_name': 'Minds Eyes..."


In [106]:
items.drop(columns=['user_url'],inplace=True)

In [131]:
items.iloc[0,3]

{'item_id': '10',
 'item_name': 'Counter-Strike',
 'playtime_forever': 6,
 'playtime_2weeks': 0}

In [108]:
items_expanded = pd.json_normalize(items['items'])

In [109]:
items_expanded

,item_id,item_name,playtime_forever,playtime_2weeks
0,10,Counter-Strike,6.0,0.0
1,20,Team Fortress Classic,0.0,0.0
2,30,Day of Defeat,7.0,0.0
3,40,Deathmatch Classic,0.0,0.0
4,50,Half-Life: Opposing Force,0.0,0.0
...,...,...,...,...
5170010,373330,All Is Dust,0.0,0.0
5170011,388490,One Way To Die: Steam Edition,3.0,3.0
5170012,521570,You Have 10 Seconds 2,4.0,4.0
5170013,519140,Minds Eyes,3.0,3.0


In [110]:
items_expanded.isnull().sum()

item_id             16806
item_name           16806
playtime_forever    16806
playtime_2weeks     16806
dtype: int64

In [117]:
items.isnull().sum()

user_id                 0
items_count             0
steam_id                0
item_id             16806
item_name           16806
playtime_forever    16806
playtime_2weeks     16806
dtype: int64

In [112]:
items_expanded = pd.json_normalize(items['items'])
items_expanded=items_expanded.replace('',None)
items = items.join(items_expanded)
items.drop(columns=['items'],inplace=True)

In [113]:
items

,user_id,items_count,steam_id,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,10,Counter-Strike,6.0,0.0
1,76561197970982479,277,76561197970982479,20,Team Fortress Classic,0.0,0.0
2,76561197970982479,277,76561197970982479,30,Day of Defeat,7.0,0.0
3,76561197970982479,277,76561197970982479,40,Deathmatch Classic,0.0,0.0
4,76561197970982479,277,76561197970982479,50,Half-Life: Opposing Force,0.0,0.0
...,...,...,...,...,...,...,...
5170010,76561198329548331,7,76561198329548331,373330,All Is Dust,0.0,0.0
5170011,76561198329548331,7,76561198329548331,388490,One Way To Die: Steam Edition,3.0,3.0
5170012,76561198329548331,7,76561198329548331,521570,You Have 10 Seconds 2,4.0,4.0
5170013,76561198329548331,7,76561198329548331,519140,Minds Eyes,3.0,3.0


In [118]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5170015 entries, 0 to 5170014
Data columns (total 7 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int64  
 2   steam_id          object 
 3   item_id           object 
 4   item_name         object 
 5   playtime_forever  float64
 6   playtime_2weeks   float64
dtypes: float64(2), int64(1), object(4)
memory usage: 276.1+ MB
